In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tkinter import *
from tkinter import ttk
from functools import partial

def SRClose(SRNo, Outcome):

    SR = SRNo
    Outcome = Outcome
    Outcome_list = ['Customer Abandoned', 'Defect Pending', 'Duplicate', 'Enhancement Request',
                   'No fault found', 'Not Entitled', 'Resolved with Solution',
                   'Resolved with Workaround']
    
    SR_list = SR.get().split(',')
    Outcome_lst = Outcome.get().split(',')
    
    for i in range(len(SR_list)):
    
        url = 'https://support.us.aaaaaaa.com/oip/faces/secure/srm/srview/SRTechnical.jspx?action=createQueryTab&srNumber='+ SR.get() + '&rowIndex=45&listBcId=6647-3709&gridId=9d37b76e-fe63-4106-8faa-62d9cb1ca075&startIncludedRowIndex=35&calcCount=16&endOfSet=N&tempGridSort=-sub-Status'

        #driver = webdriver.Chrome()
        chrome_options = webdriver.ChromeOptions()
        prefs = {"profile.default_content_setting_values.notifications" : 2}
        chrome_options.add_experimental_option("prefs",prefs)
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)

        file = open('config.txt', 'r')
        lines = file.readlines()
        username = lines[0]
        password = lines[1]

        try:
            element = WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.ID, "sso_username"))
            )
        finally:
            time.sleep(2)
            driver.find_element_by_id("sso_username").send_keys(username)
            driver.find_element_by_id("ssopassword").send_keys(password)
            driver.find_element_by_class_name("cb41w4").click()

        try:
            element = WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.TAG_NAME, "button.xl"))
            )
        finally:
            driver.find_element_by_tag_name("button.xl").click()

        try:
            element = WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.ID, "closeReopenBtn_label"))
            )
        finally:
            time.sleep(2)
            driver.find_element_by_id("closeReopenBtn_label").click()

        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID,"mossp_srclose_iframe"))) 

        try:
            element = WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.ID, "oj-combobox-input-substatusSelect"))
            )
        finally:
            driver.find_element_by_id("oj-combobox-input-substatusSelect").send_keys(Outcome.get())
            driver.find_element_by_id('opt0').click()
            WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//*[starts-with(@id,'nextBtn')]"))).click()

        while WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[starts-with(@id,'nextBtn')]"))):
            time.sleep(2)
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//*[starts-with(@id, 'nextBtn')]")))
            except:
                break
            else:
                time.sleep(2)
                try:
                    WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//*[starts-with(@id, 'nextBtn')]"))).click()
                except:
                    break

        time.sleep(5)
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,
                                                        "//*[starts-with(@id, 'closeSrBtn')]"))).click()

        driver.close()

#SRClose()

tkWindow = Tk()  
tkWindow.geometry('400x300')  
tkWindow.title('SR# Close window')

SRNoLabel = Label(tkWindow, text="SR#").grid(row=0, column=0, padx = 10, pady = 25)
SRNo = StringVar()
SRNoEntry = Entry(tkWindow, textvariable=SRNo).grid(row=0, column=1)  

OutcomeLabel = Label(tkWindow,text="Select Outcome: ").grid(row=1, column=0, padx = 10, pady = 25)  
Outcome = StringVar()
OutcomeEntry = ttk.Combobox(tkWindow, textvariable = Outcome)

OutcomeEntry['values'] = ('Customer Abandoned', 'Defect Pending', 'Duplicate', 'Enhancement Request',
                   'No fault found', 'Not Entitled', 'Resolved with Solution',
                   'Resolved with Workaround')

OutcomeEntry.grid(column = 1, row = 1)
OutcomeEntry.current()

SRClose = partial(SRClose, SRNo, Outcome)

loginButton = Button(tkWindow, text="Close SR", command=SRClose).grid(row=4, column=1, padx = 10, pady = 25)  

tkWindow.mainloop()